In [2]:
import requests 
import copy


s = {
    "baseURL": 'http://192.168.1.140:3001/',
    'id': "5bc59e192817116e84bdd831",
    'currentPoint': False
}


def getFunc(nodes):
    for key, value in nodes.items():
        req = requests.get(s['baseURL']+'pfunction/'+value['id'])
        func = req.json()
        #print(func)
        funcs[func['_id']] = func
        gr = func['pfunction']['graph']
        if (gr != {}):
            getFunc(gr['n'])

def enrich_graph(gr1):
    gr = copy.deepcopy(gr1)
    for n in gr['n']:
        x = gr['n'][n]
        x['out'] = {}; 
        x['in'] = {};
        x['position'] = {'x': 0, 'y': 0};
    for e in gr['e']:
        if e[0] == 0:
            pass
        else:
            if e[1]  in gr['n'][str(e[0])]['out']:
                gr['n'][str(e[0])]['out'][e[1]].append([e[2], e[3]])
            else:
                gr['n'][str(e[0])]['out'][e[1]] = [[e[2], e[3]]]
        gr['n'][str(e[2])]['in'][e[3]] = [e[0], e[1]]
    add_nodesedges(funcs, gr)
    visitors = [enrich]
    positionNodes(gr, list(gr['n'].keys()), [0], visitors, 0)
    s['currentPoint'] = False
    return gr

    

def add_nodesedges(context, gr):
    added_nodes = {}
    added_edges = []
    step_out = 4000
    step_in = 3000
    for n in gr['n']:
        x = gr['n'][n]
        for idx1, value1 in enumerate(context[x['id']]['pfunction']['gapi']['outputs'], start=1):
            if (idx1 not in x['out'] and x['id'] != s['id']):
                no = { 
                    'i': step_out,  
                    'svg_id': "", 
                    'id': s['id'] + str(step_out), 
                    'in':{1: [x['i'], idx1]}, 
                    'out':{}, 
                    'position': {'x': 0, 'y':0} ,
                    'edges': []
                  }
                typing  = value1 #['type']
                funcs[s['id'] + str(step_out)]  = {"_id": s['id'] + str(step_out),
                   "pclassid":"5c95397d4212cc40afeec914",
                   "pfunction":{"signature":"io", "sources": {"python":"f : f"}, 
                    "graph":{}, "gapi":{"constant": True,"inputs":[{"name":typing['name'],"type": typing['type']}],
                    "name":"io","outputs":[{"name": typing['name'], "type": typing['type']}],"payable": False,
                    "stateMutability":"view","type":"function"},"chainids":["3"]},"tags":["pipeline","ethpm"],
                    "timestamp":""}
                added_nodes[str(step_out)]  = no
                added_edges.append([x['i'], idx1, step_out, 1])
                x['out'][idx1]= [[step_out, 1]]
                step_out= step_out+1
    for n in gr['n']:
        x = gr['n'][n]
        for idx1, value1 in enumerate(context[x['id']]['pfunction']['gapi']['inputs'], start=1):
            if (idx1 not in x['in'] and x['id'] != s['id']):
                no = { 
                    'i': step_in,  
                    'svg_id': "", 
                    'id': s['id'] + str(step_in), 
                    'in': {}, 
                    'out': {1:[[x['i'], idx1]]},
                    'position': {'x': 0, 'y':0} ,
                    'edges': []
                  }
                typing  = value1 #['type']
                funcs[s['id'] + str(step_in)]  = {"_id": s['id'] + str(step_in),
                   "pclassid":"5c95397d4212cc40afeec914",
                   "pfunction":{"signature":"io", "sources": {"python":"f : f"}, 
                    "graph":{}, "gapi":{"constant": True,"inputs":[{"name":typing['name'],"type": typing['type']}],
                    "name":"io","outputs":[{"name": typing['name'],"type": typing['type']}],"payable": False,
                    "stateMutability":"view","type":"function"},"chainids":["3"]},"tags":["pipeline","ethpm"],
                    "timestamp":""}
                added_nodes[str(step_in)]  = no
                added_edges.append([step_in, 1, x['i'], idx1])
                x['in'][idx1]= [step_in, 1]
                step_in= step_in+1
    gr['n'] = {**gr['n'], **added_nodes }
    gr['e'] = gr['e'] + added_edges


def difference(first, second):
    second = set(second)
    return [item for item in first if item not in second]

def inputNodesAreVisited(node, visited):
    for key in node['in']:
        if not (node['in'][key][0] in visited): 
            return False
    return True
    
def inputNodesAreDefined(node, defined, level):
    #print(defined, node['in'])
    if (level == 0):
        return False
    for key in node['in']:
        if node['in'][key][0] in defined:
            return True
        """if not u:
            return False;
        if not u[node['in'][key][0]]:
            return False; """
    return False

def enrich(na):
    if not s['currentPoint']: 
        s['currentPoint'] = {'x': 0, 'y': 1}
    s['currentPoint']['y'] = na['level']
    if (na['row'] == 0):
        s['currentPoint']['x'] = 0
    na['node']['position'] = copy.copy(s['currentPoint'])
    max1  = max( len(na['context'][na['node']['id']]['pfunction']['gapi']['inputs']), 
                  len(na['context'][na['node']['id']]['pfunction']['gapi']['outputs']))
    s['currentPoint']['x'] = s['currentPoint']['x']  + max1
    
def positionNodes(gr, unvisited, visited, visitors, level):
    if len(unvisited) == 0:
        return;
    row = 0
    visitedNow = []
    only_outputs = True
    for unvis in unvisited:
        if int(unvis) < 4000:
            only_outputs = False
    for n in unvisited:
        if (inputNodesAreVisited(gr['n'][n], visited) or inputNodesAreDefined(gr['n'][n], gr['r'], level)) or (level >0 or int(n) > 2000):
            for visitor in visitors:
                visitor({"node": gr['n'][n],  "level": level, "row": row, "context": funcs})
            row = row + 1
            if only_outputs or int(n) < 4000:
                visitedNow.append(n)
    unvisited = difference(unvisited, visitedNow)
    visited = visited + visitedNow
    return positionNodes(gr, unvisited, visited, visitors, level+1)

def make_runtime(context, rich_graph):
    rows = {}
    rich_graph2 = copy.deepcopy(rich_graph)
    for x in rich_graph2['n']:
        if not rich_graph2['n'][x]['position']['y'] in rows:
            rows[rich_graph2['n'][x]['position']['y']] = []
        rows[rich_graph2['n'][x]['position']['y']].append(x)
    return {
    "rich_graph": rich_graph2,
    "runnable_graph": [value for (key, value) in sorted(rows.items())],
    "context": context,
    "runtime": { "0": rich_graph2['r']}
  }

def run_graph( runtime_graph, ins):
    outs = []
    ndx = 0
    arr_ins = list(ins.values())
    runtime = runtime_graph['runtime']
    runnable = runtime_graph['runnable_graph']
    rich = runtime_graph['rich_graph']
    
    for key in runnable[0]:
        runtime[key] = [arr_ins[ndx]]
        ndx= ndx+1
    ndx = 0
    for x in runnable:
        for y in runnable[ndx]:
            node  = rich['n'][y]
            contxt = runtime_graph['context'][node['id']]
            source = contxt['pfunction']['sources']['python']
            args=[]
            if  (1 in node['in']):
                port  =  0
                '''goes in order though the keys: '''
                while ( port+1 in node['in']):
                    x1 = node['in'][port+1]
                    #print(key1, x1, runtime)
                    argt = runtime[str(x1[0])][x1[1]-1]
                    type  = contxt['pfunction']['gapi']['inputs'][port]['type']
                    if (type == "function"):
                        #print(contxt['pfunction']['signature'],port+1,argt)
                        argt  = eval("lambda " + argt) 
                    port  = port +1
                    args.append(argt)
            else:
                args  = runtime[y]
            if (contxt['pfunction']['gapi']['type'] == "function"):
                if (len(contxt['pfunction']['graph']) > 0):
                    ans = run_graph (make_runtime (indexed_func) (enrich_graph (indexed_func) (contxt['pfunction']['graph']))) (args);
                    ans = ans.values()
                else:
                    func = eval("lambda " + source);
                    #print(args)
                    ans = func(*args);
            else:
                pass
            runtime[y] = [ans]
            if int(y) >=  4000:
                outs.append(ans)
        ndx = ndx + 1
    #print (runtime)
    return outs;

def fun_graph( context, graph, ins):
    return run_graph (make_runtime (context, enrich_graph (graph)),ins)


def get_graph(id):
    req = requests.get(s['baseURL']+'pfunction/'+id)
    func = req.json()
    gr = func['pfunction']['graph']
    return gr
#enrich_graph(graph)
#print(funcs, graph)

#print(make_runtime (funcs, enrich_graph (graph)))



In [3]:
graph1 = {"n":{"101":{"i":101,"id":"5dcadd175427e54a5fe18e88"},"102":{"i":102,"id":"5dcac5c45427e54a5fe18e86"},"103":{"i":103,"id":"5dcfebb36a2b0f10e93b09cb"},"104":{"i":104,"id":"5dcfec8b6a2b0f10e93b09cc"},"105":{"i":105,"id":"5dcaf0cd5427e54a5fe18e8a"},"106":{"i":106,"id":"5dcaf0cd5427e54a5fe18e8a"},"107":{"i":107,"id":"5dbbf356f18ff7488e9b1096"},"108":{"i":108,"id":"5dbbf356f18ff7488e9b1096"},"109":{"i":109,"id":"5dcadd175427e54a5fe18e88"},"110":{"i":110,"id":"5dcac5c45427e54a5fe18e86"}},"e":[[101,1,102,1],[102,1,103,1],[103,1,104,1],[102,1,105,1],[104,1,105,3],[102,1,106,1],[104,1,106,2],[103,1,106,3],[105,1,107,2],[106,1,108,2],[107,1,109,1],[108,1,109,2],[109,1,110,1]],"r":[]}
#graph1 = {"n":{"101":{"i":101,"id":"5dcadd175427e54a5fe18e88"},"102":{"i":102,"id":"5dcac5c45427e54a5fe18e86"},"103":{"i":103,"id":"5dcfebb36a2b0f10e93b09cb"},"104":{"i":104,"id":"5dcfec8b6a2b0f10e93b09cc"},"105":{"i":105,"id":"5dcaf0cd5427e54a5fe18e8a"},"106":{"i":106,"id":"5dcaf0cd5427e54a5fe18e8a"}},"e":[[101,1,102,1],[102,1,103,1],[103,1,104,1],[102,1,105,1],[104,1,105,3],[102,1,106,1],[104,1,106,2],[103,1,106,3]],"r":[]}
#graph1 = get_graph("5dcaa55a3f65440d349525f2")
#graph1 = get_graph("5dcb277d3f65440d349525f3")
#graph1 = {"n":{"101":{"i":101,"id":"5dcadd175427e54a5fe18e88"},"102":{"i":102,"id":"5dcac5c45427e54a5fe18e86"},"103":{"i":103,"id":"5dcfebb36a2b0f10e93b09cb"},"104":{"i":104,"id":"5dcfec8b6a2b0f10e93b09cc"},"105":{"i":105,"id":"5dcaf0cd5427e54a5fe18e8a"}},"e":[[101,1,102,1],[102,1,103,1],[103,1,104,1],[102,1,105,1],[104,1,105,3]],"r":[]}
print(graph1)

funcs = {}
getFunc(graph1['n'])
#print(funcs)
e = enrich_graph(graph1)
print(e)
r  = make_runtime(funcs, e)
#print(r)
test1 = {"a":[3,5,88,9,10], "b": [5,8,12,45,30,100],  "c": 0, "d":"h:h*2", "e": "h:h*3"}
run = run_graph(r , test1)

{'n': {'101': {'i': 101, 'id': '5dcadd175427e54a5fe18e88'}, '102': {'i': 102, 'id': '5dcac5c45427e54a5fe18e86'}, '103': {'i': 103, 'id': '5dcfebb36a2b0f10e93b09cb'}, '104': {'i': 104, 'id': '5dcfec8b6a2b0f10e93b09cc'}, '105': {'i': 105, 'id': '5dcaf0cd5427e54a5fe18e8a'}, '106': {'i': 106, 'id': '5dcaf0cd5427e54a5fe18e8a'}, '107': {'i': 107, 'id': '5dbbf356f18ff7488e9b1096'}, '108': {'i': 108, 'id': '5dbbf356f18ff7488e9b1096'}, '109': {'i': 109, 'id': '5dcadd175427e54a5fe18e88'}, '110': {'i': 110, 'id': '5dcac5c45427e54a5fe18e86'}}, 'e': [[101, 1, 102, 1], [102, 1, 103, 1], [103, 1, 104, 1], [102, 1, 105, 1], [104, 1, 105, 3], [102, 1, 106, 1], [104, 1, 106, 2], [103, 1, 106, 3], [105, 1, 107, 2], [106, 1, 108, 2], [107, 1, 109, 1], [108, 1, 109, 2], [109, 1, 110, 1]], 'r': []}


ConnectionError: HTTPConnectionPool(host='192.168.1.140', port=3001): Max retries exceeded with url: /pfunction/5dcadd175427e54a5fe18e88 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1060d8890>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [4]:
#print(enrich_graph (graph1))
#print(make_runtime (funcs, enrich_graph (graph1)))
#print(graph1)
#print(fun_graph(funcs, graph1, {"d": [89, 8, 89]}) ) # "a": "a, b : a + b", 
#print(fun_graph(funcs, graph1, {"array1": [2, 6, 4, 8], "array2": [9, 3, 7, 1], "start": 2, "stop": 6} ) )

test1 = {"a":[3,5,88,9,10], "b": [5,8,12,45,30,100],  "c": 0, "d":"h:h*2", "e": "h:h*3"}
print(fun_graph(funcs, graph1, test1 ) )

KeyError: '5dcadd175427e54a5fe18e88'

In [5]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import json

egraph = make_runtime (funcs, enrich_graph (graph1))
#print(egraph)


    
def operate(gr):
    inputs = []
    outputs = []
    consts = []
    def execute_gr(v):
        ins = {}
        ind = 0
        for item in inputs:
            #print(item.value)
            try:
                i = json.loads("["+item.value+"]")[0]
            except:
                i = json.loads("[\""+item.value+"\"]")[0]
            ins[str(ind)] = i
            ind = ind +1
        #print(ins)
        #outs = fun_graph(funcs, gr, ins)
        try:
            outs = run_graph(gr, ins)
        except:
            outs =[]
        ind  = 0
        for item in outputs:
            #print(item.value)
            #ins[ind] = json.loads(item.value)
            try:
                item.value = str(outs[ind])
            except:
                pass
            ind = ind +1
    for key in gr['runnable_graph']:
        for key2 in key:
            #print(key2)
            port = funcs[gr['rich_graph']['n'][key2]['id']]['pfunction']['gapi']['outputs'][0]
            if int(key2) > 2999 and int(key2) <4000:
                #node
                #print(key2)
                tinput = widgets.Text(
                    value='',
                    placeholder='Input: '+port['type'],
                    description= port['name']+': '+port['type'],
                    disabled=False
                )
                tinput.observe(execute_gr, names='value')
                inputs.append(tinput)
            if int(key2) > 3999:
                outputs.append(widgets.Text(
                    value='',
                    placeholder='Output: '+port['type'],
                    description= port['name']+': '+port['type'],
                    disabled=True
                ))
    for key in gr['runtime']:
        #print(gr['runtime'][key])
        consts.append(widgets.Text(
                    value = str(gr['runtime'][key]),
                    description='Constant:',
                    disabled=True
                )
        )
    display(widgets.HBox([widgets.VBox(inputs + consts), widgets.VBox(outputs)]))
    
operate(egraph)

KeyError: '5dcadd175427e54a5fe18e88'

In [6]:
print(egraph)


NameError: name 'egraph' is not defined